In [1]:
%matplotlib widget
%config InlineBackend.figure_format = "svg"

import addict
import copy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from importlib import reload
from tqdm.notebook import tqdm
from typing import List, Dict, Tuple

import celeri
celeri = reload(celeri)

plt.rcParams["text.usetex"] = False # Plotting the global model is much much faster with tex fonts turned off

# Read in data files, create storage dictionaries, and do basic processing

In [2]:
command_file_name = "./data/western_north_america/basic_command.json"
command, segment, block, meshes, station, mogi, sar = celeri.read_data(command_file_name)
station = celeri.process_station(station, command)
segment = celeri.process_segment(segment, command, meshes)
sar = celeri.process_sar(sar, command)
closure, block = celeri.assign_block_labels(segment, station, block, mogi, sar)
assembly = addict.Dict()
operators = addict.Dict()
operators.meshes = [addict.Dict()] * len(meshes)
assembly = celeri.merge_geodetic_data(assembly, station, sar) # Not sure this works correctly

# Get elastic operators and TDE smoothing operators

In [3]:
# Get all elastic operators for segments and TDEs
celeri.get_elastic_operators(operators, meshes, segment, station, command)

# Get TDE smoothing operators
celeri.get_all_mesh_smoothing_matrices(meshes, operators)
celeri.get_all_mesh_smoothing_matrices_simple(meshes, operators)

# Calculate non-elastic operators

In [4]:
operators.rotation_to_velocities = celeri.get_block_rotation_operator(station)
operators.global_float_block_rotation = celeri.get_global_float_block_rotation_operator(station)
assembly, operators.block_motion_constraints = celeri.block_constraints(assembly, block, command)
assembly, operators.slip_rate_constraints = celeri.slip_rate_constraints(assembly, segment, block, command)
operators.rotation_to_slip_rate = celeri.get_fault_slip_rate_partials(segment, block)
operators.block_strain_rate_to_velocities, strain_rate_block_index = celeri.get_strain_rate_centroid_operator(block, station, segment)
operators.mogi_to_velocities = celeri.get_mogi_operator(mogi, station, command)

# Plot input data

In [5]:
celeri.plot_input_summary(segment, station, block, meshes, mogi, sar, lon_range=(225, 250), lat_range=(30, 52), quiver_scale=1e2)

AttributeError: module 'celeri' has no attribute 'plot_input_summary'

# Sketching out the assembly of the block model system

$$
\begin{bmatrix}
    \mathrm{geodetic \; velocities} \\
    \mathrm{plate \; rotation \; constraints} \\
    \mathrm{slip \; rate \; constraints} \\
    \mathrm{TDE \; smoothing \; pseudodata = 0} \\
    \mathrm{TDE \; rate \; constraints} \\
    \mathrm{InSAR \; LOS \; changes} 
\end{bmatrix}
=
\begin{bmatrix}
    \mathrm{(rotations-elastic \; segments) \; to \; velocities} & \mathrm{TDEs \; to \; velocities} & \mathrm{block \; strain \; rate \; to \; velocities} & \mathrm{Mogi \; to \; velocities}\\
    \mathrm{identities}                                          & 0                                   & 0                                & 0\\
    \mathrm{plate \; rotations \; to \; slip \; rates}           & 0                                   & 0                                & 0\\
    0                                                            & \mathrm{TDE \; smoothing \; matrix} & 0                                & 0\\
    0                                                            & \mathrm{identities}                 & 0                                & 0\\
    \mathrm{(rotations-elastic \; segments) \; to \; LOS}        & \mathrm{TDEs \; to \; LOS}          & \mathrm{block \; strain \; rate \; to \; velocities}  & \mathrm{Mogi \; to \; LOS}
\end{bmatrix}
\begin{bmatrix}
    \mathrm{plate \; rotation \; rates} \\
    \mathrm{TDE \; slip \; rates} \\
    \mathrm{block \; strain \; rates} \\
    \mathrm{Mogi \; sources} 
\end{bmatrix}
$$

# Block model with block rotations, fully locked segments and partially locked subduction zone using the full tde_matrix and smoothing matrix

In [30]:
# TODO: What should this conversion be?
DEG_PER_MYR_TO_RAD_PER_YR = np.deg2rad(1) / 1e6
DEG_PER_MYR_TO_RAD_PER_YR = 1 / 1e6

# Create dictionary to store indices and sizes for operator building
index = addict.Dict()
index.n_stations = assembly.data.n_stations
index.vertical_velocities = np.arange(2, 3 * index.n_stations, 3)
index.n_blocks = len(block)
index.n_block_constraints = assembly.data.n_block_constraints
index.station_row_keep_index = celeri.get_keep_index_12(3 * len(station))
index.start_station_row = 0
index.end_station_row = 2 * len(station)
index.start_block_col = 0
index.end_block_col = 3 * len(block)
index.start_block_constraints_row = index.end_station_row
index.end_block_constraints_row = index.start_block_constraints_row + 3 * index.n_block_constraints
index.n_slip_rate_constraints = assembly.data.slip_rate_constraints.size
index.start_slip_rate_constraints_row = index.end_block_constraints_row
index.end_slip_rate_constraints_row = index.start_slip_rate_constraints_row + index.n_slip_rate_constraints

index.meshes = [addict.Dict()] * len(meshes)
index.n_tde_total = 0
for i in range(len(meshes)):
    index.meshes[i].n_tde = meshes[i].n_tde
    index.n_tde_total += index.meshes[i].n_tde
    if i == 0:
        index.meshes[i].start_tde_col = index.end_block_col
        index.meshes[i].end_tde_col = index.meshes[i].start_tde_col + 2 * index.meshes[i].n_tde
        index.meshes[i].start_tde_smoothing_row = index.end_slip_rate_constraints_row
        index.meshes[i].end_tde_smoothing_row = index.meshes[i].start_tde_smoothing_row + 2 * index.meshes[i].n_tde
    else:
        index.meshes[i].start_tde_col = index.meshes[i - 1].end_tde_col
        index.meshes[i].end_tde_col = index.meshes[i].start_tde_col + 2 * index.meshes[i].n_tde
        index.meshes[i].start_tde_smoothing_row = index.meshes[i - 1].end_tde_smoothing_row
        index.meshes[i].end_tde_smoothing_row = index.meshes[i].start_tde_smoothing_row + 2 * index.meshes[i].n_tde

index.start_tde_col = index.end_block_col
index.end_tde_col = index.start_tde_col + 2 * index.meshes[0].n_tde
index.start_tde_smoothing_row = index.end_slip_rate_constraints_row
index.end_tde_smoothing_row = index.start_tde_smoothing_row + 2 * index.meshes[0].n_tde

# Initialize data vector
estimation = addict.Dict()
estimation.data_vector = np.zeros(2 * index.n_stations + 3 * index.n_block_constraints + index.n_slip_rate_constraints + 2 * index.n_tde_total)

# Add GPS stations to data vector
estimation.data_vector[index.start_station_row:index.end_station_row] = celeri.interleave2(assembly.data.east_vel, assembly.data.north_vel)

# Add block motion constraints to data vector
estimation.data_vector[index.start_block_constraints_row:index.end_block_constraints_row] = DEG_PER_MYR_TO_RAD_PER_YR * assembly.data.block_constraints

# Add slip rate constraints to data vector
estimation.data_vector[index.start_slip_rate_constraints_row:index.end_slip_rate_constraints_row] = assembly.data.slip_rate_constraints

# Initialize and build weighting matrix
estimation.weighting_vector = np.ones(2 * index.n_stations + 3 * index.n_block_constraints + index.n_slip_rate_constraints + 2 * index.n_tde_total)
estimation.weighting_vector[index.start_station_row:index.end_station_row] = celeri.interleave2(station.east_sig, station.north_sig)
estimation.weighting_vector[index.start_block_constraints_row:index.end_block_constraints_row] = 1.0
estimation.weighting_vector[index.start_slip_rate_constraints_row:index.end_slip_rate_constraints_row] = command.slip_constraint_weight * np.ones(index.n_slip_rate_constraints)

# TODO: No need to calculate this anymore because the weighting vector approach works!!!  Show Jack first
estimation.weighting_matrix = np.diag(estimation.weighting_vector)
print(estimation.weighting_matrix.shape)

# Initialize linear operator
estimation.operator = np.zeros((2 * index.n_stations + 3 * index.n_block_constraints + index.n_slip_rate_constraints + 2 * index.n_tde_total,
                                3 * index.n_blocks + 2 * index.n_tde_total))

# Insert block rotations and elastic velocities from fully locked segments
operators.rotation_to_slip_rate_to_okada_to_velocities = operators.slip_rate_to_okada_to_velocities @ operators.rotation_to_slip_rate
estimation.operator[index.start_station_row:index.end_station_row, index.start_block_col:index.end_block_col] = operators.rotation_to_velocities[index.station_row_keep_index, :] - operators.rotation_to_slip_rate_to_okada_to_velocities[index.station_row_keep_index, :]

# Insert block motion constraints
estimation.operator[index.start_block_constraints_row:index.end_block_constraints_row, index.start_block_col:index.end_block_col] = operators.block_motion_constraints

# Insert slip rate constraints
estimation.operator[index.start_slip_rate_constraints_row:index.end_slip_rate_constraints_row, index.start_block_col:index.end_block_col] = operators.slip_rate_constraints

# Insert TDE to velocity matrix
for i in range(len(meshes)):
    # Insert TDE to velocity matrix
    tde_keep_row_index = celeri.get_keep_index_12(operators.meshes[i].tde_to_velocities.shape[0])
    tde_keep_col_index = celeri.get_keep_index_12(operators.meshes[i].tde_to_velocities.shape[1])
    estimation.operator[index.start_station_row:index.end_station_row, index.meshes[i].start_tde_col:index.meshes[i].end_tde_col] = operators.meshes[i].tde_to_velocities[tde_keep_row_index, :][:, tde_keep_col_index]

    # Insert TDE smoothing matrix
    smoothing_keep_index = celeri.get_keep_index_12(operators.meshes[i].smoothing_matrix.shape[0])
    estimation.operator[index.meshes[i].start_tde_smoothing_row:index.meshes[i].end_tde_smoothing_row, index.meshes[i].start_tde_col:index.meshes[i].end_tde_col] = meshes[i].smoothing_weight * operators.meshes[i].smoothing_matrix.toarray()[smoothing_keep_index, :][:, smoothing_keep_index]

# Solve the overdetermined linear system
estimation.state_covariance_matrix = np.linalg.inv(estimation.operator.T @ estimation.weighting_matrix @ estimation.operator)
estimation.state_vector = estimation.state_covariance_matrix @ estimation.operator.T @ estimation.weighting_matrix @ estimation.data_vector

# Solve the overdetermined linear system using only a weighting vector rather than matrix
test_state_covariance_matrix = np.linalg.inv(estimation.operator.T * estimation.weighting_vector @ estimation.operator)
test_state_vector = test_state_covariance_matrix @ estimation.operator.T * estimation.weighting_vector @ estimation.data_vector
print(np.allclose(estimation.state_vector, test_state_vector))

celeri.post_process_estimation(estimation, operators, station, index)

(7058, 7058)
True


# Plot model summary 

In [18]:
celeri.plot_estimation_summary(segment, station, estimation, lon_range=(225, 250), lat_range=(30, 52), quiver_scale=1e2)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
operator_eps = copy.deepcopy(estimation.operator) + 1e-10
plt.figure(figsize=(10, 10))
plt.imshow(np.log10(np.abs(operator_eps)), cmap="plasma_r")
plt.colorbar()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
plt.figure()
# plt.plot(np.log10(np.abs(estimation.data_vector)), "rx")
plt.plot(estimation.data_vector, "rx")
plt.show()
print(estimation.data_vector.shape)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(7058,)


In [10]:
plt.figure(figsize=(10, 5))
plt.subplot(1, 3, 1)
plt.title("strike-slip constraints")
plt.plot(segment.ss_rate_flag, "rx")
plt.subplot(1, 3, 2)
plt.title("dip-slip constraints \n should this be tensile?")
plt.plot(segment.ds_rate_flag, "bs")
plt.subplot(1, 3, 3)
plt.title("tensile-slip constraints \n should this be dip slip?")
plt.plot(segment.ts_rate_flag, "go")
plt.show()
segment.keys()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Index(['name', 'lon1', 'lat1', 'lon2', 'lat2', 'dip', 'res', 'other3',
       'other6', 'other7', 'other8', 'other9', 'other10', 'other11', 'other12',
       'locking_depth', 'locking_depth_sig', 'locking_depth_flag', 'dip_sig',
       'dip_flag', 'ss_rate', 'ss_rate_sig', 'ss_rate_flag', 'ds_rate',
       'ds_rate_sig', 'ds_rate_flag', 'ts_rate', 'ts_rate_sig', 'ts_rate_flag',
       'burial_depth', 'burial_depth_sig', 'burial_depth_flag',
       'resolution_override', 'resolution_other', 'patch_file_name',
       'patch_flag', 'patch_slip_file', 'patch_slip_flag', 'length', 'x1',
       'y1', 'z1', 'x2', 'y2', 'z2', 'mid_lon_plate_carree',
       'mid_lat_plate_carree', 'mid_lon', 'mid_lat', 'mid_x', 'mid_y', 'mid_z',
       'centroid_x', 'centroid_y', 'centroid_z', 'centroid_lon',
       'centroid_lat', 'west_labels', 'east_labels'],
      dtype='object')

In [11]:
segment.name[256]

'cfm_san_andreas_mojave_extruded_trace_part2_sa'

In [27]:
%timeit a = estimation.operator.T @ estimation.weighting_matrix
%timeit b = estimation.operator.T * estimation.weighting_vector
print(np.allclose(a, b))

1.59 s ± 130 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
54 ms ± 410 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
True


In [23]:
a

array([[-1.93236368e+01, -2.80260518e+02, -5.81853833e+02, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 1.35631068e+02,  6.33290168e+01,  7.64921481e+02, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 2.09370970e+01,  6.44057351e+01,  1.30526921e+03, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       ...,
       [ 4.33128204e-08,  7.11458419e-08,  1.67129499e-07, ...,
        -1.39989790e-01,  0.00000000e+00,  4.67444372e-02],
       [-8.03407719e-09, -5.39740892e-08, -2.86782732e-08, ...,
         0.00000000e+00, -1.18631086e-01,  0.00000000e+00],
       [ 4.34869610e-08,  6.74296987e-08,  1.68111687e-07, ...,
         7.65405239e-02,  0.00000000e+00, -1.18631086e-01]])